In [1]:
import pandas as pd

In [2]:
import re
# Access and edit Google Sheets by gspread
import gspread

In [3]:
# Module to transform gsheets to data frame
import gspread_dataframe as gs_to_df

In [4]:
from oauth2client.service_account import  ServiceAccountCredentials

In [5]:
import datetime as dt

In [6]:
from pathlib import *

In [7]:
import sys

In [8]:
path = PurePath('__file__')
sys.path.insert(0, str(Path(path.parent).resolve().parent))

In [9]:
from cpm import functions as f

In [ ]:
TEMPLATE = "Feedback_Template"

In [10]:
MATRICULA = "3. Planilha Matrículas 2019 - 1o sem"

In [30]:
MATR_ABA = "João XXIII"

In [12]:
MATR_CLEANED = "J23_Matrícula_2019-1S"

In [13]:
def check_feedback(gc, name):

    aloc = gc.open(name)

    # Convert gsheet to df
    aloc = gs_to_df.get_as_dataframe(aloc, dtype=str)


    # Transform String Dates to datetime
    f = lambda x : dt.datetime.strptime(x, "%d/%m/%Y")

    aloc['Data'] = aloc['Data'].map(f)

    # correct 'nan' strings to ''
    aloc.replace('nan', '', inplace=True)

In [14]:
def split_date_hour(col):

    return pd.Series(col.split(" "))

In [15]:
def concat_names(x,y):

    return x + " " + y

In [16]:
def split_celfone(col):

    if type(col) == str:

        pattern = ".*\(.*(\d{2})\).*(\d{5})(\d{4}).*"

        split = re.split(pattern, col)

        if len(split) >= 4:

            return "(" + split[1] + ")" + " "  + split[2] + "-" + split[3]

        return col

    return col

In [17]:
def split_fone(col):

    if type(col) == str:

        pattern = ".*\(.*(\d{2})\).*(\d{4}|\d{4})(\d{4}).*"

        split = re.split(pattern, col)

        if len(split) >= 4:

            return "(" + split[1] + ")" + " "  + split[2] + "-" + split[3]

        return col

    return col

In [37]:
def preprocess_df(df):

    presencial = df["Data/Hora preenchimento"] == "Presencial"

    espera = df["Data/Hora preenchimento"] == "Lista de Espera"

    pre = df[~ presencial & ~ espera]["Data/Hora preenchimento"]

    data_hora = pre.apply(split_date_hour)

    data = pd.Series.append(df[presencial]["Data/Hora preenchimento"],
                            df[espera]["Data/Hora preenchimento"])

    data = data.append(data_hora.iloc[:, 0]).sort_index()

    hora = pd.Series.append(df[presencial]["Data/Hora preenchimento"],
                            df[espera]["Data/Hora preenchimento"])

    hora = hora.append(data_hora.iloc[:, 1]).sort_index()

    df.rename(columns={"Data/Hora preenchimento": "Data_Pré_Matrícula"},
              inplace=True)

    df["Data_Pré_Matrícula"] = data

    df["Hora_Pré_Matrícula"] = hora

    df["Nome"] = df["Nome"].apply(str.upper).apply(str.strip)

    df["Sobrenome"] = df["Sobrenome"].apply(str.upper).apply(str.strip)

    df["Nome Responsável"] = df["Nome Responsável"].apply(str.upper).apply(str.strip)

    df["Sobrenome Responsável"] = df["Sobrenome Responsável"].apply(str.upper).apply(str.strip)

    df["Nome Responsável"] = concat_names(df["Nome Responsável"],
                                          df["Sobrenome Responsável"])

    del df["Sobrenome Responsável"]

    df["Nome"] = concat_names(df["Nome"], df["Sobrenome"])

    del df["Sobrenome"]

    df.rename(columns={"Telefone Celular ex: (011) 00000-0000": "Tel_Celular"},
              inplace=True)

    df["Tel_Celular"] = df["Tel_Celular"].apply(split_celfone)

    df.rename(columns={"Telefone Fixo ex: (011) 000-0000": "Tel_Fixo"},
              inplace=True)

    df["Tel_Fixo"] = df["Tel_Fixo"].apply(split_fone)

    df.rename(columns={"Celular do Responsável": "Celular_Responsável"},
              inplace=True)

    df["Celular_Responsável"] = df["Celular_Responsável"].apply(split_celfone)

    df.rename(columns={"RG \n(apenas números)" : "RG"}, inplace=True)

    return df

In [38]:
def main():

    gc = f.authenticate()
    
    wb = f.load_workbooks_from_drive()[MATRICULA]
    
    df = f.load_sheet_from_workbook(wb, MATR_ABA, skiprows=[1,2])[1]

    df = df.fillna('')

    df = preprocess_df(df)

    df.to_csv("matricula.csv", sep=",", index=False, columns=COLUNAS, na_rep='')

    df = pd.read_csv("matricula.csv", dtype=str, na_values='')

    matricula = gc.open(MATR_CLEANED)

    wks = matricula.worksheet("JoãoXXIII")

    wks.clear()

    gs_to_df.set_with_dataframe(worksheet=wks, dataframe=df)

In [39]:
main()

KeyError: 'Data/Hora preenchimento'

In [26]:
planilhas

{'3. Planilha Matrículas 2019 - 1o sem': <Spreadsheet '3. Planilha Matrículas 2019 - 1o sem' id:1cVRUmHE6qUjAGY-D7c1NFBC1FpYLoTwy-1-uqdEXpSI>,
 'J23_Alunos_Consolidado': <Spreadsheet 'J23_Alunos_Consolidado' id:1zg8roK0-EFySIZivBaHkLeNcVsNJ1t41YrpMzlli6vQ>,
 'J23_Matrícula_2019-1S': <Spreadsheet 'J23_Matrícula_2019-1S' id:1l1AFqVd8hWZ_xQ0wCDRqpZl4mJxWiMPWpUbV_Yp3fo8>,
 'J23_Voluntários_Consolidado': <Spreadsheet 'J23_Voluntários_Consolidado' id:16uCaqRO3sTKkVlW7pqg1bzpWBhTJpE6VLjfPVUw-4fY>,
 'J23_2019-1S_Feedback_A1': <Spreadsheet 'J23_2019-1S_Feedback_A1' id:1Lf-k5D25Zh7qynTnLEYISZhmOf_76MeOYdRBnv3yANQ>,
 'Configuração_Planilhas': <Spreadsheet 'Configuração_Planilhas' id:1rNkW25vuRKxba2TYG2u0sZ-aSRDIixwoYbz_SaebBr4>,
 'J23_2019-1S_Feedback_J': <Spreadsheet 'J23_2019-1S_Feedback_J' id:1gVZG1nfJYH9AMdl3LYf0K7qhxpNhAM-CQcCMg5D-cMA>,
 'J23_2019-1S_Feedback_I': <Spreadsheet 'J23_2019-1S_Feedback_I' id:1HqzzlEFshiyJoOGhjlWJOYrkakQ9NnQlFTOmBP9iz5Y>,
 'J23_2019-1S_Feedback_H': <Spreadsheet 'J